# < 필요한 기본 패키지 임포트 및 환경 설정 > 

In [558]:
import pandas as pd
import numpy as np

import warnings
import platform
import matplotlib.pyplot as plt

# Warning 메세지 무시
import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

# 1. 입학초 신입생 중도탈락 예측모형 개발
## 1-1) 과거(前) 입학초신입생데이터 수집
* 엑셀(excel)에 저장되어 있는 예측모형구축용 과거 입학초신입생 원천데이터셋을 데이터프레임으로 읽어들임
* 고교학업역량이나 일부 사회심리적기초조사 항목(자신감결여, 의욕저하, 경제여건 등)에서 역코딩(reverse coding)은 반영되었다고 가정함
* 과거 입학초신입생은 21/22학년도 2개년 신입생이며, 사회심리적기초조사는 입학초에 실시한 결과이고 중도탈락여부는 당해 9월 1일 기준임 

In [560]:
# 예측모형구축용 과거 입학초신입생 원천데이터셋을 데이터프레임으로 읽어들임
rawData_org = pd.read_excel('과거_입학초신입생_원천데이터셋.xlsx', sheet_name = '예측모형구축용')

# 과거 입학초신입생 원천데이터셋을 저장하는 데이터프레임 확인
rawData_org

,학생ID,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지,중도탈락여부
0,2756,학과01,남,23,거주001,수시1차,특별전형-일반고,일반고,1.8007,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0
1,2757,학과01,남,19,거주002,수시1차,특별전형-일반고,일반고,4.8083,4.5,4.3,4.3,3.7,1.0,2.5,5.0,0
2,2758,학과01,남,23,거주003,수시1차,특별전형-일반고,일반고,3.6185,3.8,3.7,3.7,4.0,3.5,2.0,4.0,0
3,2759,학과01,남,19,거주004,수시1차,특별전형-일반고,일반고,2.0236,3.0,5.0,4.3,5.0,5.0,3.0,5.0,1
4,2760,학과01,남,20,거주005,수시1차,특별전형-일반고,일반고,3.2368,5.0,4.3,4.0,3.7,3.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4733,8121,학과30,여,25,거주003,후기편입학,편입학 1학년,일반고,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4734,8122,학과16,남,21,거주009,후기편입학,편입학 1학년,일반고,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4735,8123,학과31,여,20,거주008,후기편입학,편입학 1학년,일반고,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4736,8124,학과25,남,44,거주003,후기편입학,편입학 1학년,일반고,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


## 1-2) 예측모형구축용 데이터셋 EDA 및 전처리
(1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포를 파악<br>
(2) 원천데이터셋에 존재하는 결측치(missing value) 처리<br>
(3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악<br>
(4) 원천데이터셋의 변수가 너무 많을 경우, 분석차원의 축소(또는 제거)<br>
(5) 범주형 특성(categrical feature)에 대한 인코딩<br>

### (1-2-1) 원천데이터셋을 구성하는 각 변수(특성)의 값 분포 파악

In [561]:
# NULL값의 개수 구하기
rawData_org.isnull().sum()

학생ID        0
소속학과        0
성별          0
나이          0
거주지        17
입학전형차수      0
입학전형구분      0
출신고교유형      0
고교학업역량    214
대학만족      245
학과만족      245
진로확신      245
대학생활      245
정서안정      245
경제여건      245
주변지지      245
중도탈락여부      0
dtype: int64

In [562]:
# 수치형 컬럼 각각에 대한 건수, 평균, 표준편차, 최소값, 25분위값, 50분위값, 75분위값, 최대값 구하기
rawData_org[['나이','고교학업역량','대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지']].describe()

,나이,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지
count,4738.000000,4524.000000,4493.000000,4493.000000,4493.000000,4493.000000,4493.000000,4493.000000,4493.000000
mean,19.515407,4.421369,3.995615,4.190452,4.212953,4.292321,3.526041,3.196083,4.389272
std,3.023145,1.081218,0.735396,0.723111,0.780256,0.674788,0.955424,1.044454,0.858430
min,16.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,19.000000,3.750000,3.500000,3.700000,3.700000,4.000000,3.000000,2.500000,4.000000
50%,19.000000,4.460100,4.000000,4.300000,4.300000,4.300000,3.500000,3.000000,5.000000
75%,19.000000,5.151950,4.500000,5.000000,5.000000,5.000000,4.000000,4.000000,5.000000
max,77.000000,8.314800,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [563]:
# 범주형 컬럼 각각에 대한 값의 분포
# 소속학과별 건수 (건수 내림차순으로 정렬)
rawData_org[['소속학과','학생ID']].groupby('소속학과').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
소속학과,
학과12,264
학과10,238
학과02,228
학과20,171
학과24,170
학과15,165
학과27,162
학과08,153
학과11,148


In [564]:
# 성별 건수 (건수 내림차순으로 정렬)
rawData_org[['성별','학생ID']].groupby('성별').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
성별,
여,2646
남,2092


In [565]:
# 거주지별 건수 (건수 내림차순으로 정렬)
rawData_org[['거주지','학생ID']].groupby('거주지').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
거주지,
거주003,615
거주010,273
거주006,267
거주009,242
거주008,240
...,...
거주142,1
거주144,1
거주111,1


In [566]:
# 입학전형차수별 건수 (건수 내림차순으로 정렬)
rawData_org[['입학전형차수','학생ID']].groupby('입학전형차수').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
입학전형차수,
수시1차,2758
수시2차,1333
정시,423
정시 자율,133
산업체위탁,75
후기편입학,16


In [567]:
# 입학전형구분별 건수 (건수 내림차순으로 정렬)
rawData_org[['입학전형구분','학생ID']].groupby('입학전형구분').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
입학전형구분,
특별전형-일반고,3017
특별전형-특성화고,808
일반전형,633
기초수급자/차상위계층전형,128
산업체위탁교육전형,75
농어촌전형,35
편입학 1학년,16
연계고교전형,15
대졸자전형,11


In [568]:
# 출신고교유형별 건수 (건수 내림차순으로 정렬)
rawData_org[['출신고교유형','학생ID']].groupby('출신고교유형').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
출신고교유형,
일반고,3648
특성화고,930
검정고시,82
기타,46
산업수요맞춤형고,17
예술고,6
외고,6
체육고,2
국제고,1


In [569]:
# 중도탈락여부별 건수 (건수 내림차순으로 정렬)
rawData_org[['중도탈락여부','학생ID']].groupby('중도탈락여부').학생ID.agg(['count']).sort_values(by=['count'], ascending=False)

,count
중도탈락여부,
0,4271
1,467


### (1-2-2) 원천데이터셋에 존재하는 결측치(missing value) 처리
* 총 4,738건 중 424건이 결측치로 인한 제거 대상 건(row)으로 제거 대상 건이 크지 않고 결측치를 채움으로 인한 왜곡현상도 있을 수 있어 제거 방식을 택함

In [570]:
# 결측치 제거 (row-wise deletion)
rawData_org.dropna(inplace=True)
rawData_org

,학생ID,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지,중도탈락여부
0,2756,학과01,남,23,거주001,수시1차,특별전형-일반고,일반고,1.8007,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0
1,2757,학과01,남,19,거주002,수시1차,특별전형-일반고,일반고,4.8083,4.5,4.3,4.3,3.7,1.0,2.5,5.0,0
2,2758,학과01,남,23,거주003,수시1차,특별전형-일반고,일반고,3.6185,3.8,3.7,3.7,4.0,3.5,2.0,4.0,0
3,2759,학과01,남,19,거주004,수시1차,특별전형-일반고,일반고,2.0236,3.0,5.0,4.3,5.0,5.0,3.0,5.0,1
4,2760,학과01,남,20,거주005,수시1차,특별전형-일반고,일반고,3.2368,5.0,4.3,4.0,3.7,3.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4709,7747,학과36,남,19,거주099,수시2차,일반전형,일반고,3.8432,4.3,4.7,4.3,4.3,2.5,2.0,3.0,0
4710,7748,학과36,여,20,거주003,수시2차,일반전형,일반고,2.4028,3.5,3.7,3.0,3.3,3.5,4.5,5.0,0
4711,7749,학과36,남,19,거주041,수시2차,일반전형,일반고,3.3192,2.8,4.3,3.7,3.7,3.5,4.0,5.0,0
4725,7763,학과32,여,19,거주003,수시1차,특별전형-일반고,일반고,5.5953,5.0,5.0,5.0,4.7,4.5,3.5,5.0,1


In [571]:
# 인덱스 재설정
rawData_org.reset_index(inplace=True, drop=True)
rawData_org

,학생ID,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지,중도탈락여부
0,2756,학과01,남,23,거주001,수시1차,특별전형-일반고,일반고,1.8007,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0
1,2757,학과01,남,19,거주002,수시1차,특별전형-일반고,일반고,4.8083,4.5,4.3,4.3,3.7,1.0,2.5,5.0,0
2,2758,학과01,남,23,거주003,수시1차,특별전형-일반고,일반고,3.6185,3.8,3.7,3.7,4.0,3.5,2.0,4.0,0
3,2759,학과01,남,19,거주004,수시1차,특별전형-일반고,일반고,2.0236,3.0,5.0,4.3,5.0,5.0,3.0,5.0,1
4,2760,학과01,남,20,거주005,수시1차,특별전형-일반고,일반고,3.2368,5.0,4.3,4.0,3.7,3.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,7747,학과36,남,19,거주099,수시2차,일반전형,일반고,3.8432,4.3,4.7,4.3,4.3,2.5,2.0,3.0,0
4310,7748,학과36,여,20,거주003,수시2차,일반전형,일반고,2.4028,3.5,3.7,3.0,3.3,3.5,4.5,5.0,0
4311,7749,학과36,남,19,거주041,수시2차,일반전형,일반고,3.3192,2.8,4.3,3.7,3.7,3.5,4.0,5.0,0
4312,7763,학과32,여,19,거주003,수시1차,특별전형-일반고,일반고,5.5953,5.0,5.0,5.0,4.7,4.5,3.5,5.0,1


### (1-2-3) 원천데이터셋을 구성하는 각 변수 간의 상관 관계를 파악
* 독립변수와 종속변수(중도탈락여부)와의 상관관계 파악
 * 수치형 독립변수 : 로지스틱 회귀분석
  * 결정계수가 1에 가까울수록 계수의 절대값이 클수록 p값이 낮을수록 상관관계가 높음 (계수값이 양수이면 양의 상관관계, 음수이면 음의 상관계)
 * 범주형 독립변수 : 카이제곱 검정
  * 측정한 카이제곱통계량과 자유도를 카이제곱분포표에 적용한 결과 도출된 유의확률이 미리 설정한 유의수준(상관성의 판단기준)보다 작을 때 상관성이 있다고 판단
* 독립변수들 간의 다중공선성 파악 : 분산팽창계수
 * 다중공선성 : 하나의 독립변수가 다른 여러 개의 독립변수들로 잘 예측되는 성질
  * 분산팽창계수(VIF, Variance Inflation Factor)다)가 보통 10보다 크면 다중공선성이 있다고 판단 -> 다중공선성이 있는 독립변수는 예측모형의 최종 독립변수에서 제외

In [572]:
# 수치형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 로지스틱 회귀분석(Logistic Regression)
# features : 독립변수 데이터프레임
# dropout : 종속변수 리스트
features = rawData_org[['나이','고교학업역량','대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지']]  
dropout = rawData_org['중도탈락여부']

In [573]:
# train/test set 분리하기
from sklearn.model_selection import train_test_split  
  
X_train, X_test, y_train, y_test = train_test_split(features, dropout)

In [574]:
# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler  
  
scaler = StandardScaler()  
  
X_train_std = scaler.fit_transform(X_train)  
X_test_std = scaler.fit_transform(X_test)

In [575]:
# 로지스틱 분석 시행
import statsmodels.api as sm

logreg = sm.Logit(y_train, sm.add_constant(X_train_std)).fit()
logreg.summary()

Optimization terminated successfully.
         Current function value: 0.306927
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 중도탈락여부   No. Observations:                 3235
Model:                          Logit   Df Residuals:                     3225
Method:                           MLE   Df Model:                            9
Date:                Sun, 23 Jun 2024   Pseudo R-squ.:                 0.05486
Time:                        18:01:26   Log-Likelihood:                -992.91
converged:                       True   LL-Null:                       -1050.5
Covariance Type:            nonrobust   LLR p-value:                 1.239e-20
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -2.3490      0.066    -35.578      0.000      -2.478      -2.220
x1            -0.0257      0.071     -0.364      0.716      -0.164       0.113
x2            -0.2503      0.061     -4.106      0.000      -0.370      -0.131
x3            -0.2121      0.076     -2.778      0.005      -0.362      -0.062
x4             0.0371      0.101      0.368      0.713      -0.160       0.235
x5            -0.1665      0.091     -1.822      0.068      -0.346       0.013
x6            -0.2722      0.079     -3.430      0.001      -0.428      -0.117
x7             0.1122      0.069      1.625      0.104      -0.023       0.248
x8            -0.1330      0.065     -2.040      0.041      -0.261      -0.005
x9            -0.0209      0.065     -0.320      0.749      -0.149       0.107
==============================================================================
"""

* '나이'(x1),'고교학업역량'(x2),'대학만족'(x3),'학과만족'(x4),'진로확신'(x5),'대학생활'(x6),'정서안정'(x7),'경제여건'(x8),'주변지지'(x9)
* 결정계수(Pseudo R-squ.)는 0.04608로 모델의 설명력은 약한 편임
* coef는 각 독립변수의 계수로 '정서안정'(x7)을 제외하고는 값이 클수록 중도탈락을 하지 않은 경향성을 띰
* P-value 관점에서는 '학과만족'(x4), '주변지지'(x9)의 상대적으로 설명력이 크게 약함
* 로지스틱 분석 통계 상으로는 신뢰도 높은 강건한 모델로 보기는 어려움

In [576]:
# 로지스틱 회귀모델 생성 및 평가
from sklearn.linear_model import LogisticRegression  
  
model = LogisticRegression()  
model.fit(X_train_std, y_train)

LogisticRegression()

In [577]:
print(model.score(X_train_std, y_train))
print(model.score(X_test_std, y_test))

0.9001545595054096
0.9295644114921223


* 트레이닝셋과 테스트셋에서 정확도는 0.9 이상으로 상당히 높은 편이나 정확도만으로 모델 성능을 속단하기는 이름

In [578]:
# 범주형 독립변수와 종속변수(중도탈락여부)와의 상관성 파악을 위한 카이제곱 검정(Chi-square Validation)
# 카이제곱검정을 위한 패키지 임포트
from scipy.stats import chi2_contingency

# 소속학과와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['소속학과','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['소속학과','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['소속학과','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='소속학과')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(소속학과, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 소속학과는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(소속학과, 중도탈락여부)
Chi square: 95.35746382447442
p-value: 1.7016140585790064e-07
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 소속학과는 중도탈락여부와 상관성이 있다


In [579]:
# 성별과 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['성별','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['성별','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['성별','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='성별')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(성별, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별은 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 성별은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(성별, 중도탈락여부)
Chi square: 17.4356939751243
p-value: 2.971924936439076e-05
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 성별은 중도탈락여부와 상관성이 있다


In [580]:
# 거주지와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['거주지','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['거주지','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['거주지','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='거주지')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(거주지, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 거주지는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 거주지는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(거주지, 중도탈락여부)
Chi square: 66.13578559416896
p-value: 0.5754583418799706
 -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 거주지는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)


In [581]:
# 입학전형차수와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['입학전형차수','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['입학전형차수','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['입학전형차수','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='입학전형차수')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(입학전형차수, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 입학전형차수는 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 입학전형차수는 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(입학전형차수, 중도탈락여부)
Chi square: 39.079833809943196
p-value: 1.669372070719313e-08
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 입학전형차수는 중도탈락여부와 상관성이 있다


In [582]:
# 입학전형구분와 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['입학전형구분','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['입학전형구분','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['입학전형구분','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='입학전형구분')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(입학전형구분, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 입학전형구분은 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 입학전형구분은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(입학전형구분, 중도탈락여부)
Chi square: 34.84610129838123
p-value: 1.6148900854336863e-06
 -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 입학전형구분은 중도탈락여부와 상관성이 있다


In [583]:
# 출신고교유형과 중도탈락여부 상관성 파악
# 검증함수 활용을 위한 데이터셋 재구조화
features = rawData_org[['출신고교유형','중도탈락여부','학생ID']]
cnt_df_1 = features.loc[features['중도탈락여부']==1].groupby(['출신고교유형','중도탈락여부']).학생ID.agg(['count'])
cnt_df_0 = features.loc[features['중도탈락여부']==0].groupby(['출신고교유형','중도탈락여부']).학생ID.agg(['count'])
cnt_df = pd.merge(cnt_df_1,cnt_df_0, how='inner',on='출신고교유형')
cnt_df.rename(columns={'count_x':'탈락','count_y':'지속'}, inplace=True)

# 검증함수 수행
chiresult = chi2_contingency(cnt_df, correction=False)
print('(출신고교유형, 중도탈락여부)')
print('Chi square: {}'.format(chiresult[0]))
print('p-value: {}'.format(chiresult[1]))
if chiresult[1] <= 0.05 :  # 유의수준 설정은 사용자정의인데, 대체로 0.05로 설정
    print(' -> 유의확률(p-value)이 유의수준(0.05) 이하이므로 출신고교유형은 중도탈락여부와 상관성이 있다')
else :
    print(' -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 출신고교유형은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)')

(출신고교유형, 중도탈락여부)
Chi square: 5.390031481316125
p-value: 0.2495662354496021
 -> 유의확률(p-value)이 유의수준(0.05) 초과이므로 출신고교유형은 중도탈락여부와 상관성이 없다(독립변수에서 제외 검토)


In [584]:
# 분산팽창계수를 구하기 위한 패키지(모듈) 임포트
from statsmodels.stats.outliers_influence import variance_inflation_factor
        
# 다중공선성 파악을 위한 분산팽창계수 구하기
data = rawData_org[['나이','고교학업역량','대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler  

# 정규화(표준화) 수행
scaler = StandardScaler()  
data_std = scaler.fit_transform(data)  

# VIF 출력을 위한 데이터 프레임 형성
vif = pd.DataFrame()

# VIF 값과 각 Feature 이름에 대해 설정
vif["VIF Factor"] = [variance_inflation_factor(data_std, i) for i in range(data_std.shape[1])]
vif["features"] = data.columns 

# VIF 값이 높은 순으로 정렬
vif = vif.sort_values(by="VIF Factor", ascending=False)
vif = vif.reset_index().drop(columns='index')
vif

,VIF Factor,features
0,3.092147,학과만족
1,2.705802,진로확신
2,2.065703,대학생활
3,1.795925,대학만족
4,1.333651,주변지지
5,1.317274,정서안정
6,1.151225,경제여건
7,1.069753,고교학업역량
8,1.010899,나이


* 각 독립변수에 대해서 분산팽창계수가 10 미만이므로 모두 유의미한 독립변수로 간주할 수 있음

### (1-2-4) 분석차원의 축소(또는 제거)
* 특성 선택(feature selection) : 여러 특성(변수) 중에서 종속변수에 영향도가 높은 일부 특성군을 선택하여 최종 독립변수로 활용
 * 로지스틱 회귀분석 결과 p값이 작고 계수(coef)의 절대값이 큰 변수 위주로 선택 가능 
  * 본 예제에서 고교학업역량'(x2),'대학만족'(x3),'대학생활'(x6),'정서안정'(x7) 등이 우선 고려 대상
* 특성 추출(feature extraction) : 여러 특성(변수)를 합성하여 저차원의 새로운 특성을 추출해서 최종 독립변수로 활용
  * 대표적으로 주성분분석(PCA) 기법이 있음

In [585]:
# 주성분분석
# 주성분분석을 위한 수치형 독립변수 선택 (신입생기초조사를 통해 얻은 결과로 한점)
data = rawData_org[['대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지']]

# 데이터 정규화(표준화) : 각 독립변수의 scale이 달라서 생길 수 있는 종속변수에 끼칠 영향도의 차이를 없애기 위해 동일 scale로 표준화 진행
from sklearn.preprocessing import StandardScaler  

# 정규화(표준화) 수행
scaler = StandardScaler()  
data_std = scaler.fit_transform(data)  

features = ['대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지']
pd.DataFrame(data_std, columns=features)

# 주성분분석을 위한 패키지(모듈) 임포트
from sklearn.decomposition import PCA

# 주성분 모형변수
pca = PCA(n_components=7)

# 원속성을 주성분으로 변환
printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['1', '2', '3','4','5','6','7'])

# 각 주성분의 설명력 출력 (설명력의 누적값을 활용하여 주성분의 갯수 결정. 대체로 90~95% 이상에서 결정)
print(pca.explained_variance_ratio_)

[0.47568829 0.17093334 0.10202552 0.08931665 0.07907691 0.05253728
 0.03042202]


* 설명력(설명가능한 분산량)이 0.9 이상이 되려면 5개의 주성분(PC) 필요
 * 0.47568829 + 0.17093334 + 0.10202552 + 0.08931665 + 0.07907691 = 0.91704071

In [586]:
# 주성분을 5개로 결정 (설명력 : 91.7%)
pca = PCA(n_components=5) 

printcipalComponents = pca.fit_transform(data_std)
principalDf = pd.DataFrame(data=printcipalComponents, columns = ['PC1', 'PC2', 'PC3', 'PC4', 'PC5'])
principalDf

,PC1,PC2,PC3,PC4,PC5
0,-2.911574,1.518106,0.134403,0.382742,-0.100323
1,0.547404,-2.276168,-1.176051,1.376973,-0.515179
2,1.252468,-0.536974,0.454543,-0.737233,-0.211493
3,-1.107992,0.705794,-0.254047,-1.695686,0.936128
4,-0.184854,-0.715990,-0.459546,0.464127,-0.800981
...,...,...,...,...,...
4309,0.453697,-1.575036,1.477271,0.719860,0.394125
4310,1.542285,1.379545,-1.469302,0.178190,-0.428102
4311,0.944499,0.847024,-1.510856,-0.215110,0.725885
4312,-2.366161,0.142526,0.233472,-0.099553,-0.006528


* 7개의 원속성이 5개의 주성분으로 변환되어 분석차원 축소효과를 거둠 (설명력 : 약 91.7%)

In [587]:
# 각 주성분(PC) 값을 결정하는데 있어서 원속성들의 영향력 파악 (주성분의 특성 파악을 위해 필요)
pca_df = pd.DataFrame(pca.components_, columns=['대학만족','학과만족','진로확신','대학생활','정서안정','경제여건','주변지지'])
pca_df.index = ['PC1','PC2','PC3','PC4','PC5']
pca_df

,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지
PC1,-0.409949,-0.470755,-0.450632,-0.445473,-0.265027,-0.145001,-0.342858
PC2,-0.155694,-0.183950,-0.199694,0.007940,0.588799,0.741122,-0.077693
PC3,0.323568,0.033892,-0.023179,0.239145,0.288479,-0.248771,-0.831457
PC4,0.285584,0.104579,0.161930,-0.107969,-0.635685,0.602135,-0.320881
PC5,-0.554350,0.410923,0.549312,-0.360884,0.146874,-0.007056,-0.265021


* PC1과 PC5는 대학만족, 학과만족, 진로확신, 대학생활 특성을 상대적으로 많이 반영
* PC2와 PC4는 정서안정, 경제여건 특성을 상대적으로 많이 반영
* PC3는 주변지지 특성을 월등하게 많이 반영
* -> 주성분 별로 서로 다른 특성들이 반영되어 특징을 구분할 수 있으나, 원속성의 특성이 조금씩은 반영되어 있어서 주성분으로 예측모형을 구축할 경우 예측 신뢰성과는 별도로 독립변수의 해석이 다소 모호할 수 있음

### (1-2-5) 범주형 특성(categrical feature)에 대한 인코딩
* 범주형 특성에 대해서 예측모형구축용과 예측용 데이터셋에서 동일한 범주값이 동일하게 인코딩됨을 보장하기 위해서 예측용 데이터셋도 미리 읽어들여 NULL값을 가진 건은 삭제하고 난 다음 예측모형구축용과 예측용 데이터셋을 한꺼번에 인코딩함

In [588]:
# 예측용 현 입학초신입생 원천데이터셋을 데이터프레임으로 읽어들임
# rawData_org : 예측모형구축용 원천데이터셋, rawData2_org : 예측용 원천데이터셋
rawData2_org = pd.read_excel('현재_입학초신입생_원천데이터셋.xlsx', sheet_name = '예측용')

rawData_org['구분'] = '예측모형구축용'
rawData2_org['구분'] = '예측용'

# 예측용 데이터셋에 대한 결측치 제거 (row-wise deletion)
rawData2_org.dropna(inplace=True)
rawData2_org.reset_index(inplace=True, drop=True)  # 인덱스 재설정

rawData_all = pd.concat([rawData_org, rawData2_org])

# 범주형 특성에 대한 레이블 인코딩(label encoding)
categorical_columns = ['소속학과', '성별', '거주지', '입학전형차수', '입학전형구분', '출신고교유형', '중도탈락여부']

In [589]:
# 레이블인코딩을 위한 패키지(모듈) 임포트
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in categorical_columns:
    rawData_all[feature] = le.fit_transform(rawData_all[feature]) 

# rawData_e : 전처리된 예측모형구축용 데이터셋, rawData2_e : 전처리된 예측용 데이터셋
rawData_e = rawData_all[rawData_all['구분'] == '예측모형구축용'].iloc[:, :-1]
rawData2_e = rawData_all[rawData_all['구분'] == '예측용'].iloc[:, :-1]

## 1-3) 예측모형 설계·구축
(1) 독립변수와 종속변수의 확정<br>
(2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리<br>
(3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소<br>
(4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택<br>
(5) 독립변수 별 중도탈락예측 영향도 파악<br>

### (1-3-1) 독립변수와 종속변수의 확정
* 독립변수의 개수(차원수)가 지나치게 많다면 예측모형의 성능이 떨어지고 과적합(Overfitting)의 문제가 생길 수 있음
* -> 로지스틱회귀, 카이제곱검정, 공선성분석 등을 통해 독립적이면서 종속변수에 영향도가 높은 변수들을 선택하거나 주성분분석과 같이 차원축소 방식을 통해 독립변수의 개수를 줄일 필요 있음

In [620]:
# 독립변수+종속변수 확정
# 독립변수 : 소속학과,성별,나이, 거주지, 입학전형차수, 입학전형구분, 출신고교유형, 고교학업역량, 
#            대학만족, 학과만족, 진로확신, 대학생활, 정서안정, 경제여건, 주변지지
# 종속변수 : 중도탈락여부

# feature_set_shrink : EDA를 통해 유의미성이 낮거나 종속변수와의 상관성이 낮은 독립변수를 제거할 것인지 여부 (1:제거, 0:제거하지않음)
# 유의미성이 낮은 변수 : 나이, 진로확신, 주변지지
# 상관성이 낮은 변수 : 거주지, 출신고교유형
feature_set_shrink = 0

if feature_set_shrink == 1:
    feature_list = ['소속학과', '성별', '나이', '입학전형차수', '입학전형구분', '고교학업역량', 
                              '대학만족', '진로확신', '대학생활', '정서안정', '경제여건', '중도탈락여부']
else:
    feature_list = ['소속학과', '성별', '나이', '거주지', '입학전형차수', '입학전형구분', '출신고교유형', '고교학업역량', 
                              '대학만족', '학과만족', '진로확신', '대학생활', '정서안정', '경제여건', '주변지지', '중도탈락여부']
    
analData = rawData_e[feature_list]
analData

,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지,중도탈락여부
0,0,0,23,0,0,6,5,1.8007,5.0,5.0,5.0,5.0,5.0,5.0,5.0,0
1,0,0,19,1,0,6,5,4.8083,4.5,4.3,4.3,3.7,1.0,2.5,5.0,0
2,0,0,23,2,0,6,5,3.6185,3.8,3.7,3.7,4.0,3.5,2.0,4.0,0
3,0,0,19,3,0,6,5,2.0236,3.0,5.0,4.3,5.0,5.0,3.0,5.0,1
4,0,0,20,4,0,6,5,3.2368,5.0,4.3,4.0,3.7,3.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4309,35,0,19,97,1,3,5,3.8432,4.3,4.7,4.3,4.3,2.5,2.0,3.0,0
4310,35,1,20,2,1,3,5,2.4028,3.5,3.7,3.0,3.3,3.5,4.5,5.0,0
4311,35,0,19,40,1,3,5,3.3192,2.8,4.3,3.7,3.7,3.5,4.0,5.0,0
4312,31,1,19,2,0,6,5,5.5953,5.0,5.0,5.0,4.7,4.5,3.5,5.0,1


### (1-3-2) 분석대상데이터셋을 트레이닝셋과 테스트셋으로 분리
* 전처리된 데이터셋을 독립변수 데이터셋과 종속변수 데이터셋으로 분리
* 트레이닝셋과 테스트셋 비율을 7:3으로 하여 분석대상데이터셋 분리

In [621]:
# 예측모형구축
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = analData.loc[:, feature_list[:-1]]  # 독립변수 데이터셋
y = analData.loc[:, '중도탈락여부']  # 종속변수 데이터셋

# 트레이닝셋과 테스트셋 분리에 관련된 모듈 임포트
from sklearn.model_selection import train_test_split
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
# X_train : 트레이닝셋의 독립변수 데이터셋, y_train : 트레이닝셋의 종속변수 데이터셋
# X_test : 테스트셋의 독립변수 데이터셋, y_test : 테스트셋의 종속변수 데이터셋
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### (1-3-3) 종속변수인 중도탈락여부 클래스의 분포 불균형 해소
* ∙ 중도탈락여부 변수값은 중도탈락(1)과 학업유지(0) 등 2가지 종류가 있는데, ‘학업유지’ 값을 갖는 경우가 ‘중도탈락’ 값을 갖는 경우보다 월등히 많아 분류성능지표(정확도 등)를 왜곡할 수 있음
* 이를 해결하기 위해서 적은 분포를 갖는 클래스의 투플을 인공적으로 만들어 클래스 분포의 균형을 이루는 오버샘플링(oversampling) 기법을 적용함. 대표적으로 SMOTE(Synthetic Minority Oversampling Technique) 기법이 있음
 * SMOTE(Synthetic Minority Oversampling Technique) : 데이터의 개수가 적은 클래스의 표본을 가져온 뒤 임의의 값을 추가하여 새로운 샘플을 만들어 데이터에 추가하는 오버샘플링 방식

In [622]:
# SMOTE기법(Over-sampling) 적용을 위한 패키지(모듈) 임포트
from imblearn.over_sampling import SMOTE

# 트레이닝셋에 대한 SMOTE 적용
smote = SMOTE(random_state=0)
X_train_over,y_train_over = smote.fit_resample(X_train,y_train)

print('SMOTE 적용 전 트레이닝셋의 행열 갯수: ', X_train.shape, y_train.shape)
print('SMOTE 적용 후 트레이닝셋의 행열 갯수: ', X_train_over.shape, y_train_over.shape)
print('SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: \n', pd.Series(y_train_over).value_counts())

# 테스트셋에 대한 SMOTE 적용
X_test_over,y_test_over = smote.fit_resample(X_test,y_test)
print('SMOTE 적용 전 테스트셋의 행열 갯수: ', X_test.shape, y_test.shape)
print('SMOTE 적용 후 테스트셋의 행열 갯수: ', X_test_over.shape, y_test_over.shape)
print('SMOTE 적용 후 테스트셋의 종속변수 값 분포: \n', pd.Series(y_test_over).value_counts())

SMOTE 적용 전 트레이닝셋의 행열 갯수:  (3019, 15) (3019,)
SMOTE 적용 후 트레이닝셋의 행열 갯수:  (5494, 15) (5494,)
SMOTE 적용 후 트레이닝셋의 종속변수 값 분포: 
 1    2747
0    2747
Name: 중도탈락여부, dtype: int64
SMOTE 적용 전 테스트셋의 행열 갯수:  (1295, 15) (1295,)
SMOTE 적용 후 테스트셋의 행열 갯수:  (2338, 15) (2338,)
SMOTE 적용 후 테스트셋의 종속변수 값 분포: 
 1    1169
0    1169
Name: 중도탈락여부, dtype: int64


* SMOTE 적용 후 트레이닝셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 2,747건으로 동일함
* SMOTE 적용 후 테스트셋의 1(중도탈락)군과 0(학업유지)군의 건수 분포는 각각 1,169건으로 동일함

### (1-3-4) 트레이닝셋을 대상으로 분류예측알고리듬 별로 학습을 수행하고 성능 측정/비교하여 가장 우수한 알고리듬 선택
* 비교 대상 분류예측알고리듬 : 의사결정트리(decision tree), 랜덤포레스트(random forest), XGBoost, LightGBM

In [623]:
# 분류예측 성능측정을 위한 패키지(모듈) 임포트
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 트레이닝셋 학습 및 테스트셋 검증
def f_model_fit(cls, model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)   # 트레이닝셋 학습
    pred = model.predict(X_test)  # 테스트셋 검증
    f_metrics(cls, y_test, pred)  # 성능 측정
    
# 분류예측 성능측정
def f_metrics(cls, y_test, pred):
    accuracy = accuracy_score(y_test, pred)    # 정확도(accuracy) 측정
    precision = precision_score(y_test, pred)  # 정밀도(precision) 측정
    recall = recall_score(y_test, pred)        # 재현율(recall) 측정
    f1 = f1_score(y_test, pred)                # f1-score 측정
    auc = roc_auc_score(y_test, pred)          # auc 측정
    print(cls + '정확도 : {0:.3f}, 정밀도 : {1:.3f}, 재현율 : {2:.3f}, f1-score : {3:.3f}, AUC : {4:.3f}'.format(accuracy, precision, recall, f1, auc))
    
# 분석기법 : 의사결정트리(Decision_tree) 
from sklearn.tree import DecisionTreeClassifier   # 의사결정트리 기법에 관련된 모듈 임포트

# DecisionTreeClassifier() : 의사결정트리 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# criterion : 노드 분할을 위한 품질측정방법 (gini(default), log_loss, entropy 중 택1)
# max_depth : 트리의 최대 깊이 (default : None)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
decision_tree = DecisionTreeClassifier(criterion='entropy', max_depth=4, random_state=0)
f_model_fit('Decision_tree(SMOTE적용 전) -> ', decision_tree, X_train, X_test, y_train, y_test)
f_model_fit('Decision_tree(SMOTE적용 후) -> ', decision_tree, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : 랜덤포레스트(Random_forest Classification)
from sklearn.ensemble import RandomForestClassifier  # 랜덤포레스트 기법에 관련된 모듈

# RandomForestClassifier() : 랜덤포레스트 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# random_state : 의사결정트리 생성 시 활용하는 데이터 패턴 지정. None(default)이면 트리를 만들 때마다 활용 데이터패턴이 달라져
# 결과가 달라지며, 특정 숫자를 준다면 활용 데이터패턴이 일정하여 동일한 결과를 냄
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
f_model_fit('Random_forest(SMOTE적용 전) -> ', random_forest, X_train, X_test, y_train, y_test)
f_model_fit('Random_forest(SMOTE적용 후) -> ', random_forest, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : XGBoost
import xgboost as xgb # XGBoost 기법에 관련된 모듈

# XGBClassifier() : XGBoost 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# max_depth : 트리의 최대 깊이(default:6)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# learning_rate : 학습 단계별로 이전의 결과 가중치를 얼만큼 사용할지 결정
# eval_metric : 오차 계산 방식 (rmse, mae, logloss, error, merror, mlogloss, auc)
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05, eval_metric='logloss')
f_model_fit('XGBoost(SMOTE적용 전) -> ', gbm, X_train, X_test, y_train, y_test)
f_model_fit('XGBoost(SMOTE적용 후) -> ', gbm, X_train_over, X_test_over, y_train_over, y_test_over)
print('\n')

# 분석기법 : LightGBM
from lightgbm import LGBMClassifier  # LightGBM 기법에 관련된 모듈

# LGBMClassifier() : LightGBM 모형설정 함수
# 하이퍼패러미터(hyper parameter)
# n_estimators : 모델에서 사용할 트리 갯수(학습시 생성할 트리 갯수)
# n_jobs : 학습을 위해 사용할 스레드 개수(-1은 모든 스레드를 사용함을 의미)
# boost_from_average : 불균형한 데이터 세트에서 예측 성능이 매우 저조할 경우 False로 설정 필요
lgb = LGBMClassifier(n_estimators=1000,num_leaves=64,n_jobs=-1,boost_from_average=False)
f_model_fit('LightGBM(SMOTE적용 전) -> ', lgb, X_train, X_test, y_train, y_test)
f_model_fit('LightGBM(SMOTE적용 후) -> ', lgb, X_train_over, X_test_over, y_train_over, y_test_over)

Decision_tree(SMOTE적용 전) -> 정확도 : 0.899, 정밀도 : 0.308, 재현율 : 0.032, f1-score : 0.058, AUC : 0.512
Decision_tree(SMOTE적용 후) -> 정확도 : 0.737, 정밀도 : 0.739, 재현율 : 0.735, f1-score : 0.737, AUC : 0.737


Random_forest(SMOTE적용 전) -> 정확도 : 0.902, 정밀도 : 0.333, 재현율 : 0.008, f1-score : 0.016, AUC : 0.503
Random_forest(SMOTE적용 후) -> 정확도 : 0.865, 정밀도 : 0.974, 재현율 : 0.749, f1-score : 0.847, AUC : 0.865


XGBoost(SMOTE적용 전) -> 정확도 : 0.897, 정밀도 : 0.231, 재현율 : 0.024, f1-score : 0.043, AUC : 0.508
XGBoost(SMOTE적용 후) -> 정확도 : 0.939, 정밀도 : 0.993, 재현율 : 0.884, f1-score : 0.935, AUC : 0.939


LightGBM(SMOTE적용 전) -> 정확도 : 0.893, 정밀도 : 0.158, 재현율 : 0.024, f1-score : 0.041, AUC : 0.505
LightGBM(SMOTE적용 후) -> 정확도 : 0.909, 정밀도 : 0.988, 재현율 : 0.828, f1-score : 0.901, AUC : 0.909


* 오버샘플링(SMOTE)을 적용한 결과가 그렇지 않은 결과에 비해 전반적으로 우수하고, 특히 재현율과 정밀도에 있어서 큰 효과를 보임
* 4개의 알고리듬 중에는 XGBoost가 가장 우수한 성능을 보임. 특히, 주요하게 봐야할 지표인 재현율(실제 중도탈락한 학생이 중도탈락으로 예측된 비율)에서 최고의 성능을 보였으며 AUC 값도 가장 우수함
* 그러므로, 성능측정 결과 XGBoost를 분류예측 알고리듬으로 채택하는 것이 바람직함

### (1-3-5) 독립변수 별 중도탈락예측 영향도 파악
* 중도탈락위험률 산정에 각 독립변수들이 어느 정도 영향을 미쳤는지에 대한 정량화된 수치로 중도탈락 예방을 위한 학생 관리 및 지원 프로그램에 대한 시사점(insight) 도출에 활용
* 성능측정 결과로 채택된 알고리듬 XGBoost를 기준으로 영향도가 큰 독립변수부터 출력

In [624]:
# 속성(feature) 별 영향도를 저장하는 데이터프레임 생성 
feature_imp_xgboost = pd.DataFrame({'영향도(xgboost)' : gbm.feature_importances_}, index = feature_list[:-1])
feature_imp_dtree = pd.DataFrame({'영향도(dtree)' : decision_tree.feature_importances_}, index = feature_list[:-1])
feature_imp_rforest = pd.DataFrame({'영향도(rforest)' : random_forest.feature_importances_}, index = feature_list[:-1])
feature_imp_lgbm = pd.DataFrame({'영향도(lgbm)' : lgb.feature_importances_}, index = feature_list[:-1])

feature_imp = pd.merge(feature_imp_xgboost, feature_imp_dtree, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_rforest, left_index=True, right_index=True, how='inner')
feature_imp = pd.merge(feature_imp, feature_imp_lgbm, left_index=True, right_index=True, how='inner')
feature_imp.sort_values('영향도(xgboost)', ascending=False)

,영향도(xgboost),영향도(dtree),영향도(rforest),영향도(lgbm)
성별,0.277924,0.305241,0.073029,581
주변지지,0.180502,0.000000,0.108588,1413
대학생활,0.136162,0.501270,0.138850,2892
진로확신,0.084413,0.102147,0.098085,2504
학과만족,0.067327,0.037558,0.088835,2522
출신고교유형,0.049051,0.000000,0.033113,622
경제여건,0.048592,0.000000,0.075804,3020
정서안정,0.045533,0.000000,0.068598,2833
입학전형구분,0.034815,0.040465,0.033733,619
대학만족,0.016665,0.013318,0.070297,3247


* XGBoost를 기준으로 가장 영향도가 큰 독립변수는 성별로 나타남 -> 중도탈락위험률이 상대적으로 높은 남학생들을 위한 지원프로그램 강화 모색 필요 (예, 군복무 기간 중 관리방안 등)
* 독립변수 별 중도탈락 영향도는 분류·예측 알고리듬에 따라 다르게 나타날 수 있기 때문에 절대적으로 해석하기 보다 참고정보로 활용하는 것이 바람직함. 다만, 여러 알고리듬에서 공통적으로 양향도가 높게 나타난다면, 충분히 유의미할 수 있기 때문에 중요 영향변수로 간주하고 학생지원 프로그램 설계에 반영할 필요 있음

# 2. 예측대상 데이터 수집 및 예측모형 적용
## 2-1) 현(現) 입학초 신입생 데이터 수집
* 데이터전처리 레이블인코딩 파트에서 NULL값을 가진 로우를 제거하고 범주형 특성을 레이블인코딩한 예측용 현 입학초신입생 데이터셋 활용
* 고교학업역량이나 일부 사회심리적기초조사 항목(자신감결여, 의욕저하, 경제여건 등)에서 역코딩(reverse coding)은 반영되었다고 가정함
* 현 입학초신입생은 2023학년도 신입생 기준이고 사회심리적기초조사는 입학초에 실시한 결과이며, 중도탈락여부는 당해 9월 1일 기준임 

## 2-2) 예측모형 적용
(1) 현(現) 입학초신입생 데이터셋을 대상으로 예측용 데이터셋 확보<br>
(2) 예측용 데이터셋을 예측모형에 적용하여 신입생별 중도탈락위험률과 중도탈락위험순위 구하기<br>
(3) 현 입학초신입생 중도탈락위험 예측결과와 실제 중도탈락여부(9/1일 기준) 비교<br>
(4) 현 입학초신입생별 중도탈락위험 예측결과를 엑셀(excel)로 저장<br>

### (2-2-1) 현(現) 입학초 신입생 데이터셋을 대상으로 예측용 데이터셋 확보
* 현(現) 신입생의 예측대상 원천데이터셋이 확보되면 예측모형 개발과정에서 적용했던 데이터전처리 과정을 거쳐 예측모형 적용대상 데이터셋을 확보함

In [625]:
# 전처리된 현 입학초신입생 데이터셋을 저장하는 데이터프레임 확인
rawData2_e

,학생ID,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지,중도탈락여부
0,10000,23,1,19,31,0,6,5,2.2288,4.3,3.3,3.0,4.7,2.0,1.0,4.0,0
1,10001,23,1,19,26,0,6,0,1.6250,3.8,4.7,4.3,3.7,1.5,1.0,3.0,1
2,10002,23,0,19,5,0,6,5,4.7432,5.0,5.0,5.0,5.0,4.0,3.0,5.0,0
3,10003,23,0,19,4,0,6,5,4.5869,3.5,3.7,4.7,4.0,2.5,2.5,5.0,1
4,10004,23,1,19,8,0,6,5,5.7202,3.0,4.7,4.3,4.3,4.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,9995,23,0,19,8,0,6,5,3.8643,4.8,4.3,4.0,4.3,2.5,1.5,3.0,1
1972,9996,23,0,19,2,0,6,5,4.7719,4.3,3.7,4.7,4.7,2.5,2.5,5.0,0
1973,9997,23,0,21,5,0,6,5,1.9231,3.0,3.7,4.0,5.0,5.0,4.5,5.0,0
1974,9998,23,1,19,1,0,6,5,4.1975,2.3,3.7,3.7,2.7,1.5,1.0,3.0,0


### (2-2-2) 예측용 데이터셋을 예측모형에 적용하여 신입생별 중도탈락위험률과 중도탈락위험순위 구하기
* 신입생 개인별 중도탈락예측 결과로 다음 항목을 도출할 수 있음
 * 신입생별 중도탈락위험률
   * 신입생별로 중도탈락 위험정도를 나타내는 백분위값
   * 예측모형의 함수를 활용하여 구할 수 있음
 * 신입생별 중도탈락위험순위
   * 학과/반별 중도탈락 위험순위 -> 순위가 높을수록 위험정도가 높으며, 반별 지도교수가 집중 지도 및 관리 신입생을 선별하는데 활용
   * 각 신입생들의 학생ID로 학번을 파악하고, 지도반별 신입생들의 중도탈락위험률을 내림차순 정렬하여 구할 수 있음

In [626]:
# 분석대상 특성리스트 (독립변수만)
if feature_set_shrink == 1:  # 차원축소한 경우
    feature_ind_list = ['소속학과', '성별', '나이', '입학전형차수', '입학전형구분', '고교학업역량', 
                              '대학만족', '진로확신', '대학생활', '정서안정', '경제여건']
else: # 차원축소하지 않은 경우
    feature_ind_list = ['소속학과', '성별', '나이', '거주지', '입학전형차수', '입학전형구분', '출신고교유형', '고교학업역량', 
                              '대학만족', '학과만족', '진로확신', '대학생활', '정서안정', '경제여건', '주변지지']

# 분석대상 특성으로 구성된 예측용 데이터셋
expData = rawData2_e[feature_ind_list]

# 예측용 데이터셋
expData

,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,진로확신,대학생활,정서안정,경제여건,주변지지
0,23,1,19,31,0,6,5,2.2288,4.3,3.3,3.0,4.7,2.0,1.0,4.0
1,23,1,19,26,0,6,0,1.6250,3.8,4.7,4.3,3.7,1.5,1.0,3.0
2,23,0,19,5,0,6,5,4.7432,5.0,5.0,5.0,5.0,4.0,3.0,5.0
3,23,0,19,4,0,6,5,4.5869,3.5,3.7,4.7,4.0,2.5,2.5,5.0
4,23,1,19,8,0,6,5,5.7202,3.0,4.7,4.3,4.3,4.0,3.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,23,0,19,8,0,6,5,3.8643,4.8,4.3,4.0,4.3,2.5,1.5,3.0
1972,23,0,19,2,0,6,5,4.7719,4.3,3.7,4.7,4.7,2.5,2.5,5.0
1973,23,0,21,5,0,6,5,1.9231,3.0,3.7,4.0,5.0,5.0,4.5,5.0
1974,23,1,19,1,0,6,5,4.1975,2.3,3.7,3.7,2.7,1.5,1.0,3.0


In [627]:
# predict() : 모형변수의 함수로서 신입생 각각에 대한 중도탈락여부 예측
y_pred_dtree = decision_tree.predict(expData)    # 의사결정트리
y_pred_rforest = random_forest.predict(expData)  # 랜덤포레스트 
y_pred_xgboost = gbm.predict(expData)            # XGBoost
y_pred_lgbm = lgb.predict(expData)               # LightGBM

# predict_proba() : 모형변수의 함수로서 신입생 각각에 대한 중도탈락위험률 예측
y_pred_prob_dtree = decision_tree.predict_proba(expData)     # 의사결정트리
y_pred_prob_rforest = random_forest.predict_proba(expData)   # 랜덤포레스트
y_pred_prob_xgboost = gbm.predict_proba(expData)             # GBoost
y_pred_prob_lgbm = lgb.predict_proba(expData)                # LightGBM

In [628]:
expData.loc[:, '중도탈락여부예측(dtree)'] = y_pred_dtree
expData.loc[:, '중도탈락위험률(dtree)'] = y_pred_prob_dtree[:,1]

expData.loc[:, '중도탈락여부예측(rforest)'] = y_pred_rforest
expData.loc[:, '중도탈락위험률(rforest)'] = y_pred_prob_rforest[:,1]

expData.loc[:, '중도탈락여부예측(xgboost)'] = y_pred_xgboost
expData.loc[:, '중도탈락위험률(xgboost)'] = y_pred_prob_xgboost[:,1]

expData.loc[:, '중도탈락여부예측(lgbm)'] = y_pred_lgbm
expData.loc[:, '중도탈락위험률(lgbm)'] = y_pred_prob_lgbm[:,1]

expData

,소속학과,성별,나이,거주지,입학전형차수,입학전형구분,출신고교유형,고교학업역량,대학만족,학과만족,...,경제여건,주변지지,중도탈락여부예측(dtree),중도탈락위험률(dtree),중도탈락여부예측(rforest),중도탈락위험률(rforest),중도탈락여부예측(xgboost),중도탈락위험률(xgboost),중도탈락여부예측(lgbm),중도탈락위험률(lgbm)
0,23,1,19,31,0,6,5,2.2288,4.3,3.3,...,1.0,4.0,0,0.314711,0,0.20,0,0.222485,0,1.892903e-05
1,23,1,19,26,0,6,0,1.6250,3.8,4.7,...,1.0,3.0,0,0.112796,0,0.25,0,0.139886,0,1.006173e-05
2,23,0,19,5,0,6,5,4.7432,5.0,5.0,...,3.0,5.0,0,0.289340,0,0.15,0,0.048554,0,3.035278e-07
3,23,0,19,4,0,6,5,4.5869,3.5,3.7,...,2.5,5.0,1,0.726338,0,0.15,0,0.101980,0,7.413547e-09
4,23,1,19,8,0,6,5,5.7202,3.0,4.7,...,3.0,5.0,0,0.112796,0,0.05,0,0.008686,0,1.103617e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1971,23,0,19,8,0,6,5,3.8643,4.8,4.3,...,1.5,3.0,1,0.726338,0,0.15,0,0.131771,0,2.449106e-05
1972,23,0,19,2,0,6,5,4.7719,4.3,3.7,...,2.5,5.0,0,0.253676,0,0.20,0,0.072528,0,2.016011e-06
1973,23,0,21,5,0,6,5,1.9231,3.0,3.7,...,4.5,5.0,0,0.298246,0,0.15,0,0.301519,0,1.315827e-01
1974,23,1,19,1,0,6,5,4.1975,2.3,3.7,...,1.0,3.0,0,0.418327,0,0.25,0,0.037258,0,1.412953e-09


### (2-2-3) 현 입학초신입생 중도탈락위험 예측결과와 실제 중도탈락여부(9/1일 기준) 비교
* 현 입학초신입생 데이터셋에 당해년도 9/1일 기준 중도탈락여부가 존재하므로 예측결과와 비교

In [629]:
# 예측용 데이터셋에 학생ID와 실제 중도탈락여부(9/1일기준) 컬럼 추가
expData['학생ID'] = rawData2_e['학생ID']
expData['중도탈락여부'] = rawData2_e['중도탈락여부']

In [630]:
# 예측결과와 실제결과가 일치하는 건수를 알고리듬별로 저장하는 변수 초기화
cnt_df = 0
cnt_rf = 0
cnt_xg = 0
cnt_lg = 0

# 일치여부 확인
for i in range(0, len(expData)) :
    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(dtree)'].values[i] :
        cnt_df += 1

    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(rforest)'].values[i] :
        cnt_rf += 1
    
    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(xgboost)'].values[i] :
        cnt_xg += 1  
    
    if expData['중도탈락여부'].values[i] == expData['중도탈락여부예측(lgbm)'].values[i] :
        cnt_lg += 1    

# 결과 출력        
print("Decision tree : 적중개수 %d, 적중률 %.2f" % (cnt_df, cnt_df / len(expData) * 100))
print("Random Forest : 적중개수 %d, 적중률 %.2f" % (cnt_rf, cnt_rf / len(expData) * 100))
print("XGBoost : 적중개수 %d, 적중률 %.2f" % (cnt_xg, cnt_xg / len(expData) * 100))
print("Light GBM : 적중개수 %d, 적중률 %.2f" % (cnt_lg, cnt_lg / len(expData) * 100))

Decision tree : 적중개수 1371, 적중률 69.38
Random Forest : 적중개수 1734, 적중률 87.75
XGBoost : 적중개수 1738, 적중률 87.96
Light GBM : 적중개수 1737, 적중률 87.90


* 근소한 차이로 XGBoost의 적중률이 가장 높음

### (2-2-4) 현 입학초신입생별 중도탈락위험 예측결과를 엑셀(excel)로 저장

In [631]:
# XlsxWriter 엔진으로 Pandas writer 객체 만들기
if feature_set_shrink == 1:  # 차원축소한 경우
    writer = pd.ExcelWriter('2023신입생별_중도탈락위험예측(차원축소)결과.xlsx')
else: # 차원축소하지 않은 경우
    writer = pd.ExcelWriter('2023신입생별_중도탈락위험예측결과.xlsx')
    
# 데이터프레임 내용을 엑셀에 저장    
expData.to_excel(writer, sheet_name='예측결과')
 
# Pandas writer 객체 닫기
writer.close()

* 저장된 엑셀에 신입생별 중도탈락위험률이 기록되어 있고 학생ID로 학번을 찾을 수 있으므로 학과별/지도반별로 중도탈락위험률의 내림차순으로 정렬된(중도탈락위험순위) 배포자료 작성 가능 